# Analise e processamento dos dados recebidos

# CONFIGURAR LOG DO SISTEMA

In [ ]:
import sys
from pathlib import Path
import logging
from datetime import datetime
from fiap import LoggerManager

# Adiciona o diretório src ao path do Python
src_path = Path.cwd().parent / 'src'
if str(src_path) not in sys.path:
	sys.path.insert(0, str(src_path))


log_path = Path.cwd().parent / 'logs'
LoggerManager(log_path=log_path, base_filename='data_processing')
# Dessa forma usando o logging padrão ele salva no arquivo
logging.info(f'{"=" * 50}')
logging.info(f'Início do processamento: {datetime.now()}')

# Transformar o arquivo xlsx em csv (apenas 2024)

In [ ]:
from fiap.utils import excel_to_csv

logging.info('Iniciando a conversão de Excel para CSV (2024)')
xlxs_path = Path.cwd().parent / 'data/xlxs/BASE DE DADOS PEDE 2024 - DATATHON.xlsx'
output_path = Path.cwd().parent / 'data/csv'
excel_to_csv(
    excel_path=xlxs_path,
    output_folder=output_path,
)

# Carregar dados de 2024

In [ ]:
import pandas as pd

# Caminho para o arquivo CSV de 2024
csv_path = Path.cwd().parent / 'data/csv/PEDE2024.csv'

logging.info(f'Lendo arquivo: {csv_path.name}')
df = pd.read_csv(csv_path)

logging.info(f'DataFrame carregado: {df.shape[0]} linhas, {df.shape[1]} colunas')

# nome das colunas em minusculo para padronizar
df.columns = df.columns.str.lower()
df.head()

# Exploração inicial dos dados

In [ ]:
logging.info(f'{"=" * 50}')
logging.info(f"Colunas do dataset: {list(df.columns)}")
logging.info(f"Tipos de dados:\n{df.dtypes}")
logging.info(f"Valores nulos:\n{df.isnull().sum()}")
logging.info(f"\nEstatísticas descritivas:\n{df.describe()}")

# Exemplo de um aluno
logging.info(f'{"=" * 50}')
logging.info("Exemplo de registro de aluno:")
aluno = df.loc[0].to_dict()
for k, v in aluno.items():
    logging.info(f"{k}: {v}")

# Limpeza de dados - Remover colunas desnecessárias

In [ ]:
# Colunas que serão removidas (ajuste conforme necessário)
unused_columns = [
    col for col in df.columns 
    if col.startswith('nome') or 
    col.startswith('data') or 
    col.startswith('ano') or 
    col.startswith('ra') or
    col.startswith('avaliador') or
    col.startswith('nº') or
    col.startswith('turma')
]

if unused_columns:
    logging.info(f"Removendo {len(unused_columns)} colunas desnecessárias: {unused_columns}")
    df = df.drop(columns=unused_columns)
    logging.info(f"DataFrame após limpeza: {df.shape[0]} linhas, {df.shape[1]} colunas")
else:
    logging.info("Nenhuma coluna desnecessária encontrada")

df.head()

In [ ]:
print(df["ativo/ inativo"].value_counts())
print(df["ativo/ inativo.1"].value_counts())

In [ ]:
# como estao todos cursando vou remover essas colunas
df = df.drop(columns=["ativo/ inativo", "ativo/ inativo.1"])

In [ ]:
logging.info(f'{"=" * 50}')
logging.info(f'Processamento concluído: {datetime.now()}')
logging.info(f'Dataset final: {df.shape[0]} linhas, {df.shape[1]} colunas')

# REMOVER VALORES NULOS

In [ ]:
# Remover colunas que tenham mais de 30% de valores nulos
threshold = len(df) * 0.3
cols_to_drop = df.columns[df.isnull().sum() > threshold]
if len(cols_to_drop) > 0:
    logging.info(f"Removendo {len(cols_to_drop)} colunas com mais de 30% de valores nulos: {list(cols_to_drop)}")
    df = df.drop(columns=cols_to_drop)
    logging.info(f"DataFrame após remoção de colunas nulas: {df.shape[0]} linhas, {df.shape[1]} colunas")

In [ ]:
df.head(1)

# ENCODING

In [ ]:
df.dtypes

In [ ]:
df.loc[1]

# Padronizar Fases

In [ ]:
logging.info(f'{"=" * 50}')
logging.info("Contagem de alunos por fase:")
for f, q in df["fase"].value_counts().items():
    logging.info(f"Fase: {f}, Quantidade: {q}")

In [ ]:
logging.info("Contagem de alunos por fase ideal:")
for f, q in df['fase ideal'].value_counts().items():
    logging.info(f"Fase ideal: {f}, Quantidade: {q}")

Vamos padronizar para ALFA e fase1-8

In [ ]:
# Padronização da coluna "fase"
def padronizar_fase(valor):
    v = str(valor).strip().lower()
    v = v.replace("fase ", "")  # Remove o prefixo "fase " se existir
    if v.startswith("alfa"):
        return "alfa"
    # Extrai o primeiro número encontrado (1-8) para as fases
    import re
    match = re.search(r"[1-8]", v)
    if match:
        return f"{match.group(0)}"
    return v

# Aplicar padronização
logging.info("Padronizar Fase")
df["fase"] = df["fase"].apply(padronizar_fase)

logging.info("Padronização de fases concluída.")
logging.info("Contagem de alunos por fase padronizada:")
for f, q in df["fase"].value_counts().items():
    logging.info(f"Fase: {f}, Quantidade: {q}")

# fase ideal
logging.info("Padronizar Fase Ideal")
df["fase ideal"] = df["fase ideal"].apply(padronizar_fase)

logging.info("Padronização de fase ideal concluída.")
logging.info("Contagem de alunos por fase ideal padronizada:")
for f, q in df["fase ideal"].value_counts().items():
    logging.info(f"Fase: {f}, Quantidade: {q}")

In [ ]:
logging.info("Remover Escola pois assim pode ser usado para novas escolas")
df.drop(columns=["escola"], inplace=True)